In [1]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd

In [ ]:
#For Specialization
spl_title=[]
spl_link=[]

import timeit

import time
start = time.time()
for i in range(1,16):
    #44&index=prod_all_products_term_optimization_v3&entityTypeDescription=Specializations&allLanguages=English
    url = "https://www.coursera.org/directory/specializations?page=" +str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    st = soup.find_all('a', class_ ="c-directory-link")
    num=len(st)
    for j in range(1,num):
        #spl-title
        st = soup.find_all('a', class_ ="c-directory-link")[j].get_text()
        spl_title.append(st)       
        #spl URL
        curl= soup.find_all("a", class_='c-directory-link')[j]
        spl_link.append("https://www.coursera.org"+ curl.get('href'))
stop = time.time()
print(f"Training time: {stop - start}s")
#creating a dataframe of all specializations
splz=pd.DataFrame({'spl_title':spl_title,
                   'spl_link':spl_link})


#writing into a .csv file
#coursera_df.to_csv('Coursera_catalog.csv')

Training time: 26.182609796524048s


In [2]:
#get all courses on coursera
course_title=[]
course_link=[]

cnt=0
for i in range(1,153):
    #44&index=prod_all_products_term_optimization_v3&entityTypeDescription=Specializations&allLanguages=English
    url = "https://www.coursera.org/directory/courses?page=" +str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    st = soup.find_all('a', class_ ="c-directory-link")
    num=len(st)
    for j in range(1,num):
        #course-title
        ct = soup.find_all('a', class_ ="c-directory-link")[j].get_text()
        cnt=cnt+1
        course_title.append(ct)       
        #course URL
        curl= soup.find_all("a", class_='c-directory-link')[j]
        course_link.append("https://www.coursera.org"+ curl.get('href'))

#creating a dataframe of all specializations
courses=pd.DataFrame({'course_title':course_title,
                   'course_link':course_link})

#writing into a .csv file
#coursera_df.to_csv('Coursera_catalog.csv')

In [ ]:
courses.course_link.describe

<bound method NDFrame.describe of 0           https://www.coursera.org/learn/makingprogress
1       https://www.coursera.org/learn/writing-for-bus...
2       https://www.coursera.org/learn/ictus-agudo-esc...
3       https://www.coursera.org/learn/brand-and-conte...
4       https://www.coursera.org/learn/cluster-analysi...
                              ...                        
5163       https://www.coursera.org/learn/jisuanji-xitong
5164      https://www.coursera.org/learn/jisuanji-zucheng
5165        https://www.coursera.org/learn/fanyi-ruanjian
5166    https://www.coursera.org/learn/lunwen-xiezuo-c...
5167    https://www.coursera.org/learn/game-theory-and...
Name: course_link, Length: 5168, dtype: object>

In [ ]:
cs=courses.iloc[[16,27,24,25]]
cs

,course_title,course_link
16,Know Thyself - The Value and Limits of Self-Kn...,https://www.coursera.org/learn/know-thyself-th...
27,Use Canva to Create Desktop and Mobile-friendl...,https://www.coursera.org/learn/use-canva-to-cr...
24,Supervised Learning: Classification (IBM),https://www.coursera.org/learn/supervised-lear...
25,System Administration and IT Infrastructure Se...,https://www.coursera.org/learn/system-administ...


In [ ]:
#To get detail of each course

course_name=[]
course_type=[]
course_photo_URL=[]
typ=[]
typ_name=[]
university_name=[]
university_logo=[]
course_link=[]
spl_skills=[]
course_tit=[]
time_required=[]
level=[]
course_language=[]
course_subtitles=[]
course_skills=[]
category=[]
sub_category=[]
start = time.time()
for urlC in courses.course_link[1222:5168]:
    page_course = requests.get(urlC)
    soup = BeautifulSoup(page_course.content, 'html.parser')
    
    guided=soup.find('div',class_="_fc5ifbq _bv93ce")
    
    if guided is None:
      #Attaching Course Link
      course_link.append(urlC)
      #Attaching course Name/Title
      cnn=soup.find('h1',class_="banner-title m-b-0 banner-title-without--subtitle").get_text()
      course_name.append(cnn)
      #Attaching type of course
      typ.append('COURSE')
      
      #attaching University /name
      univ = soup.find('h4', class_ = 'headline-4-text bold rc-Partner__title').get_text()
      university_name.append(univ)

      #attaching University photo
      u_logo=soup.find('div',class_='rc-Partner')
      l=u_logo.find('img').get('src')
      university_logo.append(l)
      
      
      try:
        

      #Attaching course level(Beginner/Intermediate/ Advanced) and  time required to complete
        fet=soup.find_all('div',class_='_y1d9czk m-b-2 p-t-1s')
        for i in range(len(fet)):
          fett=fet.find('div',class_="_16ni8zai m-b-0 m-t-1s")[i].get_text()
          if 'Level'in fett:
            level.append(fett)
          
          elif 'complete' in fett:
            time_required.append(fett)

      except:
        level.append("mixed/not-mentioned")
        time_required.append("not-mentioned")

      
      
      
      #Attaching Course Languages and course subtitles
      lang=soup.find_all("div",class_="_y1d9czk m-b-2 p-t-1s")[-1]
      lgg= lang.find("div",class_="_16ni8zai m-b-0").get_text()
      subtitless=lang.find("div",class_="content-inner").get_text()
      course_language.append(lgg)
      course_subtitles.append(subtitless)

      #Attachng course skills
      skills= [x.get_text() for x in soup.find_all('span', class_ = '_rsc0bd m-r-1s m-b-1s')]
      course_skills.append(skills)
      skills=[]

      #attaching Course Category and sub-category
      try:
        subj = soup.find_all('a',class_ = '_172v19u6 color-white font-weight-bold')
        category.append(subj[1].get_text())
        sub_category.append(subj[2].get_text())
      except IndexError:
        category.append("not-mentioned")
        sub_category.append("not-mentioned")
      
    else:
      #Attaching Course Link
      course_link.append(urlC)
      #Attaching course Name/Title
      cnn=soup.find('h1',class_="_125g251l _gkjc69").get_text()
      course_name.append(cnn)
      #Attaching type of course
      typ.append('GUIDED PROJECT')
      
      #attaching University /name
      university_name.append('Coursera Project Network')

      #attaching University photo
      u_logo=soup.find('div',class_='_1tu07i3a')
      l=u_logo.find('img').get('src')
      university_logo.append(l)
      #Attaching course level(Beginner/Intermediate/ Advanced) and  time required to complete
      try:
        lang=soup.find_all("div",class_="_8m7gipb _1f096on")[1]
        lgg= lang.find("span").get_text()
      
        level.append(lgg)
      
      #time required to complete the course
        vtim=soup.find('span',class_='_1rcyblj')
        time_required.append(vtim)
      #Attaching Course Languages and course subtitles
        lang=soup.find_all("span",class_="_1rcyblj")[-2].get_text()
        course_language.append(lang)
        course_subtitles.append(lang)

      except:
        level.append("mixed/not-mentioned")
        time_required.append("not-mentioned")
        course_language.append("not-mentioned")
        course_subtitles.append("not-mentioned")


      #Attachng course skills
      try:
        skills= [x.get_text() for x in soup.find_all('span', class_ = '_1q9sh65')]
        course_skills.append(skills)
        skills=[]
      except:
        course_skills.append("not-mentioned")
        

      #attaching Course Category and sub-category
      try:

        subj = soup.find_all('a',class_ = '_k8ap7s6')
        category.append(subj[1].get_text())
        sub_category.append(subj[2].get_text())
      except IndexError:       
        category.append("not-mentioned")
        sub_category.append("not-mentioned")
stop = time.time()
print(f"Training time: {stop - start}s")

In [ ]:
len(course_link)

6

In [ ]:
#To get detail of each course

course_name=[]
course_type=[]
course_photo_URL=[]
typ=[]
typ_name=[]
university_name=[]
university_logo=[]
course_link=[]
spl_skills=[]
course_tit=[]
time_required=[]
level=[]
course_language=[]
course_subtitles=[]
course_skills=[]
category=[]
sub_category=[]
start = time.time()
for urlC in courses.course_link:
    page_course = requests.get(urlC)
    soup = BeautifulSoup(page_course.content, 'html.parser')
    
    guided=soup.find('div',class_="_fc5ifbq _bv93ce")
    
    if guided is None:
      #Attaching course Name/Title
      cnn=soup.find('h1',class_="banner-title m-b-0 banner-title-without--subtitle")
      if cnn is None:
        continue
      else:
        cnn.get_text()
        course_name.append(cnn)


      
      #Attaching Course Link
      course_link.append(urlC)

      
      #Attaching type of course
      typ.append('COURSE')
      
      #attaching University /name
      univ = soup.find('h4', class_ = 'headline-4-text bold rc-Partner__title').get_text()
      university_name.append(univ)

      #attaching University photo
      u_logo=soup.find('div',class_='rc-Partner')
      l=u_logo.find('img').get('src')
      university_logo.append(l)
      #Attaching course level(Beginner/Intermediate/ Advanced) and  time required to complete
      
      #fet=soup.find_all('div',class_='_y1d9czk m-b-2 p-t-1s')[-1]


      #for i in range(0,2):

        #if i==0:
          #lev=fet[i].get_text()
          #level.append(lev)
        #else:
          #vtim=fet[i].get_text()
          #time_required.append(vtim)

      
      
      
      #Attaching Course Languages and course subtitles
      lang=soup.find_all("div",class_="_y1d9czk m-b-2 p-t-1s")[-1]
      lgg= lang.find("div",class_="_16ni8zai m-b-0").get_text()
      subtitless=lang.find("div",class_="content-inner").get_text()
      course_language.append(lgg)
      course_subtitles.append(subtitless)

      #Attachng course skills
      skills= [x.get_text() for x in soup.find_all('span', class_ = '_rsc0bd m-r-1s m-b-1s')]
      course_skills.append(skills)
      skills=[]

      #attaching Course Category and sub-category
      subj = soup.find_all('a',class_ = '_172v19u6 color-white font-weight-bold')
      category.append(subj[1].get_text())
      sub_category.append(subj[2].get_text())
    else:
      #Attaching Course Link
      course_link.append(urlC)
      #Attaching course Name/Title
      cnn=soup.find('h1',class_="_125g251l _gkjc69").get_text()
      course_name.append(cnn)
      #Attaching type of course
      typ.append('GUIDED PROJECT')
      
      #attaching University /name
      university_name.append('Coursera Project Network')

      #attaching University photo
      u_logo=soup.find('div',class_='_1tu07i3a')
      l=u_logo.find('img').get('src')
      university_logo.append(l)
      #Attaching course level(Beginner/Intermediate/ Advanced) and  time required to complete
      lang=soup.find_all("div",class_="_8m7gipb _1f096on")[1]
      lgg= lang.find("span").get_text()
      
      level.append(lgg)
      
      #time required to complete the course
      vtim=soup.find('span',class_='_1rcyblj')
      time_required.append(vtim)
      #Attaching Course Languages and course subtitles
      lang=soup.find_all("span",class_="_1rcyblj")[-2].get_text()
      course_language.append(lang)
      course_subtitles.append(lang)


      #Attachng course skills
      skills= [x.get_text() for x in soup.find_all('span', class_ = '_1q9sh65')]
      course_skills.append(skills)
      skills=[]

      #attaching Course Category and sub-category
      subj = soup.find_all('a',class_ = '_k8ap7s6')
      category.append(subj[1].get_text())
      sub_category.append(subj[2].get_text())
      
stop = time.time()
print(f"Training time: {stop - start}s")

IndexError: ignored

In [ ]:
len(course_link)

1216

In [ ]:
print(course_link)

['https://www.coursera.org/learn/makingprogress', 'https://www.coursera.org/learn/writing-for-business-ar', 'https://www.coursera.org/learn/ictus-agudo-escala-race', 'https://www.coursera.org/learn/brand-and-content-marketing', 'https://www.coursera.org/learn/cluster-analysis-rcmdr', 'https://www.coursera.org/learn/comercio-inmigracion-tipos-de-cambio', 'https://www.coursera.org/learn/create-a-budget-with-google-sheets', 'https://www.coursera.org/learn/enhance-organizational-communications-with-slack', 'https://www.coursera.org/learn/fpga-intro', 'https://www.coursera.org/learn/chinese-character-writing-2', 'https://www.coursera.org/learn/healthcare-payment-models', 'https://www.coursera.org/learn/it-security', 'https://www.coursera.org/learn/network-transformation-101', 'https://www.coursera.org/learn/international-labour-law', 'https://www.coursera.org/learn/introduction-g-suite', 'https://www.coursera.org/learn/javascript-while-loop', 'https://www.coursera.org/learn/know-thyself-the

In [ ]:
#Attaching Course Languages

courseraCourses=pd.DataFrame({'course_name':course_name,
'course_type':typ,
'university':university_name,
'course_link':course_link,
'course_category':category,
'course_sub_category':sub_category,
'univ_logo':university_logo,
'course_lang':lg,
'course_subtitles':subtitles,
'course_level':level,
'course_period':time_required,
'course_skills':course_skills

})

In [ ]:
courseraCourses

In [22]:
url="https://www.coursera.org/learn/uva-darden-agile-analytics"
page_course = requests.get(url)
soup = BeautifulSoup(page_course.content, 'html.parser')
#lang=soup.find_all("div",class_="_y1d9czk m-b-2 p-t-1s")
#for i,z in zip(range(len(lang)),lang):
  #g=z.find('div',class_="_16ni8zai m-b-0 m-t-1s")
  #if g is None: continue
  #else: print(g.get_text())

      #Attaching course level(Beginner/Intermediate/ Advanced) and  time required to complete
fet=soup.find_all('div',class_='_y1d9czk m-b-2 p-t-1s')
for i in fet:
      fett=i.find('div',class_="_16ni8zai m-b-0 m-t-1s")
      if fett is None:
        continue
      elif "Level" in fett.get_text():
        print(fett.get_text())
        continue  
      elif 'complete' in fett.get_text():
        print(fett.get_text())
        break
      



  




Beginner Level
Approx. 14 hours to complete
